In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
weather_station = pd.read_excel('/content/drive/MyDrive/🍟愛與羈絆的製造小組🍟/data/氣象局測站列表.xlsx')

In [ ]:
# weather_station['站號'] = pd.to_numeric(weather_station['站號'],errors='coerce')
# weather_station = weather_station[~weather_station['站號'].isna()]
# weather_station['站號'] = weather_station['站號'].astype(int).astype(str)
# weather_station

# 爬天氣資料

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import os

# 年資料
# crawler_type='Year'
# 月資料
crawler_type='Month'
# 日資料
# crawler_type='Day'



path = '/content/drive/MyDrive/🍟愛與羈絆的製造小組🍟/data/weather/all/'
for i in range(len(weather_station)):
  # now_station = ''
  now_station = weather_station.iloc[i]['站號']
  print('=====',now_station,'======')
  try:
    os.mkdir(path+str(now_station))
  except:
    print('exist')
    pass

  if crawler_type=='Day':
    date_list = list(pd.date_range(datetime(2020,10,1), datetime(2021,10,1), freq="1D").astype(str))
  if crawler_type=='Month':
    date_list = list(pd.date_range(datetime(2020,10,1), datetime(2021,10,1), freq="1M").astype(str).map(lambda x: x[:7]))
  if crawler_type=='Year':
    date_list = list(range(2017,2022))
  for date in date_list:
      date = str(date)
      print(now_station, date)

      url="https://e-service.cwb.gov.tw/HistoryDataQuery/"+crawler_type+"DataController.do?command=viewMain&station="+str(now_station)+"&stname=%25E8%2587%25BA%25E5%258C%2597&datepicker="+date

      csvfile=path+str(now_station)+'/'+date+".csv" #開個csv檔案準備寫入
      # print(url)
      r=requests.get(url)
      r.encoding="utf-8"
      soup=BeautifulSoup(r.text,"lxml")
      tag_table=soup.find(id="MyTable") #用BeautifulSoup找到table位置
      rows=tag_table.findAll("tr") #找到每個

      with open(csvfile,"w+",newline="",encoding="utf-8") as fp:
          writer=csv.writer(fp)
          i = 0
          for row in rows:
              if i>1:
                  rowList=[]
                  for cell in row.findAll(["td","th"]):
                      rowList.append(cell.get_text().replace("\n","").replace("\r","").replace("\xa0",""))
                  writer.writerow(rowList)
              i+=1

串流輸出內容已截斷至最後 5000 行。
C0M770 2021-02
C0M770 2021-03
C0M770 2021-04
C0M770 2021-05
C0M770 2021-06
C0M770 2021-07
C0M770 2021-08
C0M770 2021-09
===== C0M780 ======
C0M780 2020-10
C0M780 2020-11
C0M780 2020-12
C0M780 2021-01
C0M780 2021-02
C0M780 2021-03
C0M780 2021-04
C0M780 2021-05
C0M780 2021-06
C0M780 2021-07
C0M780 2021-08
C0M780 2021-09
===== C0M790 ======
C0M790 2020-10
C0M790 2020-11
C0M790 2020-12
C0M790 2021-01
C0M790 2021-02
C0M790 2021-03
C0M790 2021-04
C0M790 2021-05
C0M790 2021-06
C0M790 2021-07
C0M790 2021-08
C0M790 2021-09
===== C0M800 ======
C0M800 2020-10
C0M800 2020-11
C0M800 2020-12
C0M800 2021-01
C0M800 2021-02
C0M800 2021-03
C0M800 2021-04
C0M800 2021-05
C0M800 2021-06
C0M800 2021-07
C0M800 2021-08
C0M800 2021-09
===== C0M810 ======
C0M810 2020-10
C0M810 2020-11
C0M810 2020-12
C0M810 2021-01
C0M810 2021-02
C0M810 2021-03
C0M810 2021-04
C0M810 2021-05
C0M810 2021-06
C0M810 2021-07
C0M810 2021-08
C0M810 2021-09
===== C0M820 ======
C0M820 2020-10
C0M820 2020-11
C0M820 20

# Merge 所有天氣資料

In [ ]:
path = '/content/drive/MyDrive/🍟愛與羈絆的製造小組🍟/data/weather/all/'
stations = os.listdir(path)
weather_all = pd.DataFrame()

for station in stations[:]:
  print('===='+station+'====')
  files = os.listdir(path+station+'/')
  weather_station = pd.DataFrame()
  for file in files[:]:
    print(file)
    weather = pd.read_csv(path+station+'/'+file)
    weather['ObsTime']=pd.Series([file[:7]+'-']*weather.shape[0])+weather['ObsTime'].astype(str)
    weather['ObsTime'] = pd.to_datetime(weather['ObsTime'])
    weather = weather.rename(columns={'ObsTime':'time'})
    weather_station = pd.concat([weather_station, weather],ignore_index=True)
  weather_station['station']=station
  weather_all = pd.concat([weather_all, weather_station],ignore_index=True)
# weather_all['city']='center'
weather_all.to_csv('/content/drive/MyDrive/🍟愛與羈絆的製造小組🍟/data/center_all_1721.csv', index=False, encoding="utf-8-sig")